## Let The Trained Agent Roam Free!

This notebook will take our trained agent from ___ and allow it to roam free to collect Bananas. This is our AI agent in action!

In [1]:
from unityagents import UnityEnvironment
import torch
import torch.nn as nn

In [2]:
env = UnityEnvironment(file_name="Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name]
action_size = brain.vector_action_space_size
input_dims = len(env_info.vector_observations[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


#### Agents DQN spec. 
_This could have been seperated into its own .py file but I've kept it in the notebook for visibility_

In [3]:
class DuellingDeepQNetwork(nn.Module):

    def __init__(self, input_dim, output_dim, fc1_units=64, fc2_units=64):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.A = nn.Linear(fc2_units, output_dim)
        self.V = nn.Linear(fc2_units, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        A = self.A(x)
        V = self.V(x)
        return V, A

#### We Instantiate the agent and load our pre-trained weights

In [4]:
agent = DuellingDeepQNetwork(input_dims, action_size)
agent.load_state_dict(torch.load("DuelingDQN.pth"))

In [5]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    _, A = agent.forward(torch.from_numpy(state).float().unsqueeze(0))        # send state to DQN
    action = torch.argmax(A).item()                # argmax to follow determinstic poilcy
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))


Score: 13.0


In [6]:
env.close()